### Sentence Window Retriever ###
- Embed each sentence for retrieval
- Send a "window" of surrounding sentences for generation
- Supported Strategies
    - S002_00 -> Sentence Window
    - S001_01 -> Sentence Window  + Rereank

In [1]:
# Fetch API keys from config.py
import os
from config import set_environment 
set_environment()

import logging
import sys

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Only for notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.cohere_rerank import CohereRerank

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding

from datetime import datetime
import pandas as pd

Choose the model for generation

In [3]:
generation_llm_family = os.environ["GENERATION_LLM_FAMILY"]
generation_llm_model = os.environ["GENERATION_LLM_MODEL"]

if generation_llm_family == "OPENAI":
    Settings.llm = OpenAI(temperature=0, model=generation_llm_model)
elif generation_llm_family == "COHERE":
    Settings.llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=generation_llm_model,temperature=0)

Choose the model for embedding

In [4]:
embedding_llm_family = os.environ["EMBEDDING_LLM_FAMILY"]
embedding_llm_model = os.environ["EMBEDDING_LLM_MODEL"]

if embedding_llm_family == "OPENAI":
    Settings.embed_model = OpenAIEmbedding(model=embedding_llm_model,dimensions=512,)
elif embedding_llm_family == "COHERE":
    Settings.embed_model = CohereEmbedding(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model_name=embedding_llm_model,
    input_type="search_query",
)

Set the parameters for the run here

In [5]:
eval_name = os.environ["EVAL_NAME"]
eval_directory = os.environ["EVAL_DIRECTORY"]
eval_file = os.environ["EVAL_FILE"]
eval_questions = os.environ["EVAL_QUESTIONS"]
eval_results_dir = os.environ["EVAL_RESULTS_DIR"]
eval_quick_test = os.environ["EVAL_QUICK_TEST"]

rag_strategy = os.environ["RAG_STRATEGY"]

similarity_top_k = int(os.environ["SIMILARITY_TOP_K"])

# Context Post Processor Settings
similarity_cutoff = float(os.environ["SIMILARITY_CUTOFF"])

# Node Parser
window_size = int(os.environ["WINDOW_SIZE"])

Pick the strategy

In [6]:
if rag_strategy == "S002_00":
    rag_strategy_desc = "SWindow_Basic"
    run_id = f"{eval_name}_{rag_strategy}_GM_{generation_llm_model}_EM_{embedding_llm_model}_W_{window_size}_K_{similarity_top_k}_{datetime.today().strftime('%Y-%m-%d')}"
elif rag_strategy == "S002_01": 
    rag_strategy_desc = "SWindow_Rerank"
    reranker = os.environ["RERANKER"]
    rerank_top_n = int(os.environ["RERANK_TOP_N"])
    run_id = f"{eval_name}_{rag_strategy}_GM_{generation_llm_model}_EM_{embedding_llm_model}_W_{window_size}_K_{similarity_top_k}_RR_{reranker}_N_{rerank_top_n}_{datetime.today().strftime('%Y-%m-%d')}"

output_file = f"{eval_results_dir}/{run_id}.xlsx"

Set up Token Counting

In [8]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-4").encode
)

Settings.callback_manager = CallbackManager([token_counter])
tokencount_df = pd.DataFrame()

Read the documents, create chunks, calculate embeddings, store in a vector database

In [9]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=window_size,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()
Settings.text_splitter = text_splitter

documents = SimpleDirectoryReader(eval_directory).load_data()
nodes = node_parser.get_nodes_from_documents(documents)
sentence_index = VectorStoreIndex(nodes,show_progress=True)

Generating embeddings:   0%|          | 0/812 [00:00<?, ?it/s]

In [10]:
tokencount_df['document_tokens'] = [token_counter.total_embedding_token_count]
token_counter.reset_counts()

Set up retrieval and response generation

In [11]:
if rag_strategy =="S002_00":
    node_postprocessors = [
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ]
elif rag_strategy == "S002_01":
    cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=rerank_top_n)
    node_postprocessors = [
        MetadataReplacementPostProcessor(target_metadata_key="window"), cohere_rerank
    ]

query_engine = sentence_index.as_query_engine(
    similarity_top_k= similarity_top_k,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=node_postprocessors,
)


Quick test of query engine

In [12]:
response = query_engine.query(eval_quick_test)
print(f"Question:{eval_quick_test}{chr(10)}")
print(f"Response:{chr(10)}{response.response}{chr(10)}")

text_md = ""
text_wd = ""

for n in response.source_nodes:

    window = n.node.metadata["window"]
    sentence = n.node.metadata["original_text"]
    
    text_md += (
        f"**Node ID:** {n.node.node_id}{chr(10)}"
        f"**Similarity:** {n.score}{chr(10)}"
        f"**Text:** {n.node.get_content()}{chr(10)}"
        f"{chr(10)}------------------{chr(10)}"
        f"***Original Sentence***:{chr(10)} {sentence}"
        f"{chr(10)}------------------{chr(10)}"
        f"***Window***:{chr(10)}{window}{chr(10)}"
        f"~~~~{chr(10)}"
    )
      
    
print(text_md)

Question:My disabled daughter is 28 years old. Is she covered?

Response:
Your daughter would not be considered a covered family member under the ITPEU Health and Welfare Plan. According to the information provided, a covered family member is a relative who is below the age of 26, or above the age of 26 but incapable of self-support due to physical or developmental disabilities, and only if their incapacity began before they reached the age of 26. Since your daughter is 28 years old, she doesn't meet the requirements for coverage.

**Node ID:** 3f421cad-13dd-424f-a466-503ea834faa1
**Similarity:** 0.7927853
**Text:** ITPEU Benefits > Health & Welfare Plan > The Health and Welfare Plan > Part III: Plan Document
https://itpeubenefits.org/Health-Welfare-Plan/The-Health-and-Welfare-Plan/Part-III-Plan-Document[4/3/2024 1:33:35 PM]Event.
 (c) If a second Qualifying Event (for example, a death or divorce) takes place within the 18 month
maximum coverage period following the termination of empl

- Read the evalution question set (along with expected answers)
- This is structured in Llamaindex's format for batch evaluations
- Also, load into a data frame (which we will write back to an excel file with responses, evaluations etc.)

In [13]:
with open(eval_questions, 'r') as file:
    data = pd.read_json(file)
     
    queries_df = pd.DataFrame(list(data['queries'].items()), columns=['query_num', 'query'])
    responses_df = pd.DataFrame(list(data['responses'].items()), columns=['query_num', 'expected_answer'])
    
    responses_df = pd.merge(queries_df, responses_df, on='query_num')

In [14]:
from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import QueryResponseDataset

Send questions to engine in bulk

In [15]:
eval_dataset = QueryResponseDataset.from_json(eval_questions)
eval_qs = eval_dataset.questions
ref_response_strs = [r for (_, r) in eval_dataset.qr_pairs]
pred_responses = get_responses(
    eval_qs, query_engine, show_progress=True
)
pred_response_strs = [str(p) for p in pred_responses]

c:\Users\sthan\anaconda3\Lib\site-packages\llama_index\core\evaluation\dataset_generation.py:109: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return cls(**data)
100%|██████████| 6/6 [00:03<00:00,  1.65it/s]


In [16]:
from typing import List, Optional, Tuple
def get_answers_source_nodes(responses)->Tuple[list, list]:

    answers = []
    sources = []
        
    for response in responses:
        answers.append(response.response)
        text_md = ""
        
        for n in response.source_nodes:
            window = n.node.metadata["window"]
            sentence = n.node.metadata["original_text"]

            text_md += (
            f"**Node ID:** {n.node.node_id}{chr(10)}"
            f"**Similarity:** {n.score}{chr(10)}"
            f"**Text:** {n.node.get_content()}{chr(10)}"
            f"{chr(10)}------------------{chr(10)}"
            f"***Original Sentence***:{chr(10)} {sentence}"
            f"{chr(10)}------------------{chr(10)}"
            f"***Window***:{chr(10)}{window}{chr(10)}"
            f"~~~~{chr(10)}"
    )

        sources.append(text_md)
    
    return answers, sources

In [17]:
answers, sources = get_answers_source_nodes(pred_responses)

responses_df['generated_answer'] = answers

sources_df = pd.DataFrame()
sources_df['query_num'] = responses_df['query_num']
sources_df['query'] = responses_df['query']
sources_df = sources_df.join(pd.DataFrame(sources)[0].str.split("~~~~", expand=True))

tokencount_df['answer_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [18]:
with pd.ExcelWriter(output_file) as writer:
   responses_df.to_excel(writer, sheet_name="Responses", index=False)
   sources_df.to_excel(writer, sheet_name="Sources", index=False)
   tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)

Choose the LLM for evaluations

In [19]:
evaluation_llm_family = os.environ["EVALUATION_LLM_FAMILY"]
evaluation_llm_model = os.environ["EVALUATION_LLM_MODEL"]

if evaluation_llm_family == "OPENAI":
    Settings.eval_llm = OpenAI(temperature=0, model=evaluation_llm_model)
elif evaluation_llm_family == "COHERE":
    Settings.eval_llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=evaluation_llm_model, temperature=0)

Setup Evaluations

In [20]:
from llama_index.core.evaluation import QueryResponseDataset
#import random

from llama_index.core.evaluation.eval_utils import (
    get_responses,
)
from llama_index.core.evaluation import BatchEvalRunner

from llama_index.core.evaluation import (
    CorrectnessEvaluator,
)

from deepeval.integrations.llama_index import (
    DeepEvalAnswerRelevancyEvaluator,
    DeepEvalFaithfulnessEvaluator,
    DeepEvalContextualRelevancyEvaluator,
    DeepEvalBiasEvaluator,
    DeepEvalToxicityEvaluator,
)

c:\Users\sthan\anaconda3\Lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.13, however version 0.21.24 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [21]:
eval_lidx_c = CorrectnessEvaluator(llm=Settings.eval_llm)
eval_deval_f = DeepEvalFaithfulnessEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_ar = DeepEvalAnswerRelevancyEvaluator( threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_cr = DeepEvalContextualRelevancyEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_b = DeepEvalBiasEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)
eval_deval_t = DeepEvalToxicityEvaluator(threshold=0.5, model=evaluation_llm_model,include_reason=True)

In [22]:
# For large eval sets (30+ questions)
evaluator_dict_essential = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f
}

# For troubleshooting 
evaluator_dict_extended = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Context_Relevancy": eval_deval_cr
}

# For small sets (< 10 questions)
evaluator_dict_full = {
    "Correctness": eval_lidx_c,
    "Faithfulness": eval_deval_f,
    "Answer_Relevancy": eval_deval_ar,
    "Context_Relevancy": eval_deval_cr,
    "Bias": eval_deval_b,
    "Toxicity": eval_deval_t ,
}

# Pick the list of evaluators to run
evaluator_dict = evaluator_dict_essential

# Make sure this list matches the chosenevaluator_dict 
evaluators = ["Correctness", "Faithfulness" ] 

batch_runner = BatchEvalRunner(evaluator_dict, workers=8)

In [23]:
eval_results = await batch_runner.aevaluate_responses(
    queries=eval_qs,
    responses=pred_responses,
    reference=ref_response_strs,
)

Output()

Output()

Output()

Output()

Output()

Output()

In [24]:
from evaluation_utils import get_eval_results_df, get_summary_scores_df

In [25]:
mean_df, sum_df = get_summary_scores_df(
    [eval_results ],
    [rag_strategy],
    evaluators
)

In [26]:
if "Correctness" in evaluators:
    correctness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Correctness"]
    )
    responses_df['correctness'] = correctness_df['score']

if "Faithfulness" in evaluators:
    faithfulness_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Faithfulness"]
    )
    responses_df['faithfulness'] = faithfulness_df['score']

if "Answer_Relevancy" in evaluators:
    answer_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Answer_Relevancy"]
    )
    responses_df['answer_relevancy'] = answer_relevancy_df['score']

if "Context_Relevancy" in evaluators:
    context_relevancy_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Context_Relevancy"]
    )
    responses_df['context_relevancy'] = context_relevancy_df['score']

if "Bias" in evaluators:
    bias_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Bias"]
    )
    responses_df['bias'] = bias_df['score']

if "Toxicity" in evaluators:
    toxicity_df = get_eval_results_df(
        list(responses_df['query_num']),
        list(responses_df['expected_answer']),
        eval_results["Toxicity"]
    )
    responses_df['toxicity'] = toxicity_df['score']

In [27]:
responses_df['rag_strategy'] = rag_strategy
responses_df['rag_strategy_desc'] = rag_strategy_desc
responses_df['parameter_1'] = window_size
responses_df['parameter_2'] = similarity_top_k
if rag_strategy =="S002_00":
   responses_df['parameter_3'] = ""
elif rag_strategy == "S002_01":
    responses_df['parameter_3'] = rerank_top_n
responses_df['parameter_4'] = ""
responses_df['parameter_5'] = ""
responses_df['model'] = generation_llm_model 
responses_df['embed_model'] = embedding_llm_model 
responses_df['eval_model'] = evaluation_llm_model
if rag_strategy =="S002_00":
   responses_df['reranker'] = ""
elif rag_strategy == "S002_01":
    responses_df['reranker'] = reranker
responses_df['run_date'] = datetime.today().strftime('%Y-%m-%d') 
responses_df['eval_name'] = eval_name

In [28]:
tokencount_df['eval_tokens' ] = [token_counter.total_llm_token_count]
token_counter.reset_counts()

In [29]:
with pd.ExcelWriter(output_file) as writer:
    responses_df.to_excel(writer, sheet_name="Responses", index=False)
    sources_df.to_excel(writer, sheet_name="Sources", index=False)
    
    sum_df.to_excel(writer, sheet_name="Summary", index=False, startrow=0 , startcol=0)
    mean_df.to_excel(writer, sheet_name="Summary", index=False,startrow=5, startcol=0)
   
  
    if "Correctness" in evaluators:
        correctness_df.to_excel(writer, sheet_name="Correctness", index=False)
    
    if "Faithfulness" in evaluators:
        faithfulness_df.to_excel(writer, sheet_name="Faithfulness", index=False)

    if "Context_Relevancy" in evaluators:
        context_relevancy_df.to_excel(writer, sheet_name="Context_Relevancy", index=False)
    
    if "Answer_Relevancy" in evaluators:
        answer_relevancy_df.to_excel(writer, sheet_name="Answer_Relevancy", index=False)

    if "Bias" in evaluators:
        bias_df.to_excel(writer, sheet_name="Bias", index=False)

    if "Toxicity" in evaluators:
        toxicity_df.to_excel(writer, sheet_name="Toxicity", index=False)
    
    tokencount_df.to_excel(writer, sheet_name="Token Counts", index=False)